# モデルの性能を評価

In [ ]:
target_motif = "PTAP"

In [ ]:
%cd ..
%pwd

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

path = 'reports/performance_evaluation.csv'
df = pd.read_csv(path)
df

# 学習データセットに対する性能を表示

ウイルス毎に結果をまとめる

In [ ]:
def recall(group):
    return group['tp'].sum() / (group['tp'].sum() + group['fn'].sum())

def precision(group):
    return group['tp'].sum() / (group['tp'].sum() + group['fp'].sum())

df_virus = pd.DataFrame(columns=['precision', 'recall'])
df_virus['recall'] = df.groupby('virus').apply(recall)
df_virus['precision'] = df.groupby('virus').apply(precision)

df_virus

### 学習データセット数を表示

In [ ]:
path = 'reports/dataset_size.csv'
df = pd.read_csv(path)
df

ウイルス毎に学習データセット数を表示

In [ ]:
df_virus = df.groupby('virus').sum()
df_virus

# 非学習データセットに対する評価結果

In [ ]:
import json

filename = "reports/parameters.json"
with open(filename, 'r') as f:
    parameters = json.load(f)

eval_threshold = parameters['eval_threshold']

学習に用いていないウイルスのアミノ酸配列に対する出力値を取得

In [ ]:
filename = "reports/positive_pred_on_nontraining_virus.csv"
df = pd.read_csv(filename)

In [ ]:
df = df[df['output'] > eval_threshold]

df

モチーフの正規表現を満たす配列だけを抽出

In [ ]:
import json
import re


motif_re_path = "references/motif_regular_expression.json"

def apply_regular_expression(df):
    with open(motif_re_path, 'r') as f:
        motif_re = json.load(f)
        pattern = motif_re[target_motif]

    pattern = '.*' + pattern + '.*'
    pattern = re.compile(pattern)

    def pattern_match(seq):
        if pattern.match(seq):
            return True
        else:
            return False

    rows_to_drop = ~df['seq'].map(pattern_match)
    df = df.drop(df[rows_to_drop].index, inplace=True)

df = apply_regular_expression(df)
df